# 1. Build an object detection model with Amazon SageMaker's built-in Object Detection algorithm

<div class="alert alert-block alert-warning">
<b>Prerequisite:</b> This notebook is part of the <a href="https://catalog.us-east-1.prod.workshops.aws/workshops/f64d7705-d176-4775-a124-314cf5f404f8">Computer vision for retail inventory workshop</a>. Please follow the workshop instructions before running this notebook.
</div>

<div class="alert alert-block alert-warning">
<b>Notebook Kernel:</b> Please use the Python 3 (Data Science) kernel to run this notebook.
</div>

In this notebook we will build a custom object detector model, using [Amazon SageMaker's built-in Object Detection algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection.html), which is based on the Single Shot multibox Detector (SSD). The object detection model will be the main component for our retail inventory monitoring system. 

Like most of the built-in algorithms, the Object Detection documentation includes a [How It Works](https://docs.aws.amazon.com/sagemaker/latest/dg/algo-object-detection-tech-notes.html) section, with an overview and links to relevant resources. The SSD algorithm is described in [Liu et al, 2016](https://arxiv.org/pdf/1512.02325.pdf).


## Step 0: Dependencies and configuration

Start by loading useful Python libraries, defining our configuration, and connecting to the AWS SDKs, which will allow us to interface with various AWS services, like Amazon S3 and Amazon SageMaker. 

In [ ]:
# generic packages
import os
import json
import shutil
import imageio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
plt.style.use('seaborn')

# AWS-related packages
import boto3  # AWS Python SDK that allows us to interface with all AWS services
import sagemaker  # SageMaker Python SDK that allows us to easily build, train and deploy models

Next we will connect to the AWS SDKs and set up some common variables we will be using, in order to define the folder structure locally and in S3.

In [ ]:
sagemaker_role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session()
boto_session = boto3.session.Session()
region = boto_session.region_name


Amazon SageMaker provides a **default bucket** in each AWS region, which is automatically created for us. We will be using this default bucket to store our data, model artifacts and model outputs. 

In [ ]:
# variable setup
BUCKET_NAME = sagemaker_session.default_bucket()  # here we will store our data
PREFIX_PROJECT = 'computer-vision-for-retail-workshop'  # main project folder in S3
PREFIX_DATASET = 'dataset-full'  # where our dataset will be located
PREFIX_MODELS = 'models'  # where our trained model weights will be saved
CLASS_NAMES = [  # names of the 10 products that we will be trying to detect
    'flakes',  
    'mm', 
    'coke', 
    'spam', 
    'nutella', 
    'doritos', 
    'ritz', 
    'skittles', 
    'mountaindew', 
    'evian'
]
# print a list of the class names along with their indices
print('Index and class name')
class_indx_names = pd.Series(CLASS_NAMES)
print(class_indx_names,'\n')

MANIFEST_ATTRIBUTE_NAMES = ['source-ref', 'retail-object-labeling']  # attributes to be considered in the manifest files
LOCAL_DATASET_FOLDER = 'dataset'

print('Region:', region)
print('Bucket:', BUCKET_NAME)

# initialize some empty variables we need to exist:
predictor_std = None
predictor_hpo = None


## Step 1: Prepare the dataset

A small training dataset of 95 images is already included, in the file `dataset.zip`. The dataset includes images of the same shelf, with different combinations of the 10 products placed in various random positions on it. 

The dataset has already been split into 3 parts: 
- Train (66 images)
- Validation (19 images)
- Test (10 images)

In each split, the distribution of training examples is roughly similar across the 10 item classes. 3 [manifest files](https://docs.aws.amazon.com/sagemaker/latest/dg/sms-input-data-input-manifest.html) are also included: 
- `train.manifest`
- `validation.manifest`
- `test.manifest`

Each manifest file describes which of the images belong to each split, along with details about the bounding boxes of the items in each image. 

First, we will need to unzip the dataset, explore it, update the manifest files, and upload everything to the S3 bucket. 

In [ ]:
! unzip dataset/dataset-full.zip -d dataset/

Here we will show one image and its corresponding annotations from the `train.manifest` file. In a manifest file, each line is a stand-alone JSON expression, containing metadata. In our case, it contains an image filename and its annotations (i.e. bounding box coordinates and class name).

In [ ]:
# depict one image from the dataset, along with its corresponding annotations

from pathlib import Path

with open(f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/train.manifest') as f:
    lines = f.readlines()

line_dict = json.loads(lines[0])  # load the 1st line of the manifest file
filename = str(Path(line_dict['source-ref']).name)

image = imageio.imread(f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/{filename}')
plt.imshow(image)
plt.grid(False)
plt.axis(True)
plt.title(f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/{filename}')
plt.show()

print(json.dumps(line_dict, indent=4))

Next, we will copy all the image files (training + validation + testing) to the S3 bucket. We will need them to be in the S3 bucket, in order to initiate training with Amazon SageMaker.

In [ ]:
import glob

ls_dataset_files = glob.glob(f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/*.jpg')  # get all image files
print('Copying', len(ls_dataset_files), 'images to', f's3://{BUCKET_NAME}/{PREFIX_PROJECT}/{PREFIX_DATASET}')

for file in ls_dataset_files:
    filename = str(Path(file).name)
    print('Copying', filename)
    sagemaker_session.upload_data(
        path=f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/{filename}',
        bucket=BUCKET_NAME,
        key_prefix=f'{PREFIX_PROJECT}/{PREFIX_DATASET}',
    )

We now need to update the manifest files with the correct location of the images in our S3 bucket, and copy them to the same S3 location. At the same time, we will count the number of training examples per class, per split, in order to understand more about our dataset. 

In [ ]:
#---------- analyze and update the train.manifest
new_manifest = []
n_samples_training = 0
class_histogram_train = np.zeros(len(CLASS_NAMES), dtype=int)

with open(f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/train.manifest') as f:  # open the manifest file
    lines = f.readlines()
for line in lines:
    line_dict = json.loads(line)  # load one json line (corresponding to one image)
    
    filename = str(Path(line_dict['source-ref']).name)
    new_filename_s3 = f's3://{BUCKET_NAME}/{PREFIX_PROJECT}/{PREFIX_DATASET}/{filename}'
    line_dict['source-ref'] = new_filename_s3
    new_manifest.append(json.dumps(line_dict))  # add updated json line
    
    n_samples_training += 1  # counting training images
    for i,annotation in enumerate(line_dict['retail-object-labeling']['annotations']):
        class_histogram_train[int(line_dict['retail-object-labeling']['annotations'][i]['class_id'])] += 1  # counting annotations

# save the updated training manifest file locally
with open(f"{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/train-updated.manifest", "w") as f:
    for line in new_manifest:
        f.write(f"{line}\n")        
        
        
#---------- analyze and update the validation.manifest       
new_manifest = []
n_samples_validation = 0
class_histogram_val = np.zeros(len(CLASS_NAMES), dtype=int)

with open(f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/validation.manifest') as f:
    lines = f.readlines()
for line in lines:
    line_dict = json.loads(line)  # load one json line (corresponding to one image)
    
    filename = str(Path(line_dict['source-ref']).name)
    new_filename_s3 = f's3://{BUCKET_NAME}/{PREFIX_PROJECT}/{PREFIX_DATASET}/{filename}'
    line_dict['source-ref'] = new_filename_s3
    new_manifest.append(json.dumps(line_dict))  # add updated json line
    
    n_samples_validation += 1  # counting validation images
    for i,annotation in enumerate(line_dict['retail-object-labeling']['annotations']):
        class_histogram_val[int(line_dict['retail-object-labeling']['annotations'][i]['class_id'])] += 1  # counting validation samples

# save the updated validation manifest file locally
with open(f"{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/validation-updated.manifest", "w") as f:
    for line in new_manifest:
        f.write(f"{line}\n")           

        
#---------- analyze and update the test.manifest           
new_manifest = []
n_samples_testing = 0
class_histogram_test = np.zeros(len(CLASS_NAMES), dtype=int)

with open(f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/test.manifest') as f:
    lines = f.readlines()
for line in lines:
    line_dict = json.loads(line)  # load one json line (corresponding to one image)
    
    filename = str(Path(line_dict['source-ref']).name)
    new_filename_s3 = f's3://{BUCKET_NAME}/{PREFIX_PROJECT}/{PREFIX_DATASET}/{filename}'
    line_dict['source-ref'] = new_filename_s3
    new_manifest.append(json.dumps(line_dict))  # add updated json line
    
    n_samples_testing += 1  # counting testing images
    for i,annotation in enumerate(line_dict['retail-object-labeling']['annotations']):
        class_histogram_test[int(line_dict['retail-object-labeling']['annotations'][i]['class_id'])] += 1  # counting testing samples

# save the updated test manifest file locally
with open(f"{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/test-updated.manifest", "w") as f:
    for line in new_manifest:
        f.write(f"{line}\n")         
        

In [ ]:
# copy the new updated manifest files to S3

sagemaker_session.upload_data(
    path=f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/train-updated.manifest',
    bucket=BUCKET_NAME,
    key_prefix=f'{PREFIX_PROJECT}/{PREFIX_DATASET}'
)

sagemaker_session.upload_data(
    path=f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/validation-updated.manifest',
    bucket=BUCKET_NAME,
    key_prefix=f'{PREFIX_PROJECT}/{PREFIX_DATASET}'
)

sagemaker_session.upload_data(
    path=f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/test-updated.manifest',
    bucket=BUCKET_NAME,
    key_prefix=f'{PREFIX_PROJECT}/{PREFIX_DATASET}'
)

In [ ]:
# depict statistics about the existing dataset splits

df_dataset_stats = pd.DataFrame(
    {
        'train': class_histogram_train, 
        'validation': class_histogram_val, 
        'test': class_histogram_test, 
        'class': CLASS_NAMES
    }
) 

df_dataset_stats.plot.bar(x='class')
plt.title('Number of examples per split')
plt.show()


Finally, we will copy the training images into a separate local folder. This will make our life easier later when we test the performance of our model. 


In [ ]:
# initialize local test set folder
local_testset_folder = f'{LOCAL_DATASET_FOLDER}/dataset-test'  
if os.path.exists(local_testset_folder) is False:
    os.makedirs(local_testset_folder)
    
with open(f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/test-updated.manifest') as f:
    lines = f.readlines()
for i,line in enumerate(lines):
    line_dict = json.loads(line)
    filename = str(Path(line_dict['source-ref']).name)
    shutil.copy(  
        src=f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/{filename}',  # copy test images to a separate folder for later testing
        dst=f'{local_testset_folder}/{filename}'
    )

shutil.copy(  
        src=f'{LOCAL_DATASET_FOLDER}/{PREFIX_DATASET}/test-updated.manifest',  # also copy the test manifest file
        dst=f'{local_testset_folder}/test-updated.manifest'
    )

print('Test images located in:', local_testset_folder)

## Step 2: Set up input data channels

In order for SageMaker to train a machine learning model, it needs to know where the training and validation datasets are located. In SageMaker language, this is called **input channels**. Input channels are **objects** containing the location of the datasets in S3, along with the type of data and annotations. 

In our case, we have in S3, both for the training and validation datasets:

* A **JSONLines manifest file** listing what images are in the data-set (by their S3 URI) and what annotations have been collected for those images (bounding boxes + class names)
* The image files themselves

We want SageMaker to provide the algorithm with a **stream of image records** comprising both the image data and their annotations. This will be faster compared to downloading the full dataset to the training container. The [algorithm docs](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection.html#object-detection-inputoutput) give guidance on how to set this up: SageMaker already provides functionality to create RecordIO files from manifest files. 

In [ ]:
train_channel = sagemaker.inputs.TrainingInput(
    f's3://{BUCKET_NAME}/{PREFIX_PROJECT}/{PREFIX_DATASET}/train-updated.manifest',
    distribution="FullyReplicated",  # In case we want to try distributed training
    content_type="application/x-recordio",
    s3_data_type="AugmentedManifestFile",
    record_wrapping="RecordIO",
    attribute_names=MANIFEST_ATTRIBUTE_NAMES,  # focus only on specific attributes inside the manifest file
    shuffle_config=sagemaker.inputs.ShuffleConfig(seed=1)
)
                                        
validation_channel = sagemaker.inputs.TrainingInput(
    f's3://{BUCKET_NAME}/{PREFIX_PROJECT}/{PREFIX_DATASET}/validation-updated.manifest',
    distribution="FullyReplicated",  # In case we want to try distributed training
    content_type="application/x-recordio",
    record_wrapping="RecordIO",
    s3_data_type="AugmentedManifestFile",
    attribute_names=MANIFEST_ATTRIBUTE_NAMES,  # focus only on specific attributes inside the manifest file
)

## Step 3: Configure the algorithm

The first step in deciding to use a SageMaker built-in algorithm is to review its [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection.html) and [hyperparameters](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection-api-config.html). In particular we'll need the **URL for the Docker image** in order to use a built-in algorithm. While this is listed [in the docs](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html), it's also nice and easy to fetch programmatically.


In [ ]:
training_image = sagemaker.image_uris.retrieve(
    region=region, 
    framework="object-detection", 
    version="1"  # or you can use "latest"
)
print('Container image:', training_image)


The remainder of configuration includes:

* Setting where to store final model artifacts and intermediate checkpoints
* Specifying which compute resource to use
* Selecting the algorithm's hyperparameters

We do this through the [SageMaker SDK's Estimator API](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html), similarly to estimators in other common frameworks. Some things to keep in mind:

* [Pipe Mode](https://docs.aws.amazon.com/sagemaker/latest/dg/cdf-training.html#cdf-pipe-mode) streams input data to the algorithm rather than (the default) downloading the whole dataset up-front. This can accelerate training start-up for algorithms that support it.
* The Object Detection built-in algorithm supports GPU-accelerated and distributed training. Here we use a GPU-accelerated `ml.p3.2xlarge` instance. There is no need for distributed training (more than one instance),  due to the small dataset size.

In [ ]:
estimator = sagemaker.estimator.Estimator(
    training_image,  # URL to container image implementing the algorithm 
    sagemaker_role,  # IAM access to perform the API actions
    input_mode="Pipe",  # or "File" mode
    instance_count=1,  # if more than 1, then we have distributed training (not needed here!)
    instance_type="ml.p3.2xlarge",  # type of instance to be used for training
    volume_size=50,  # (in GB) storage volume to use for storing input and output data during training 
    max_run=10*60*60,  # (in sec) maximum time of the training job
    output_path=f"s3://{BUCKET_NAME}/{PREFIX_PROJECT}/{PREFIX_MODELS}"  # where to store the model weights
)


Next we will select the [algorithm's hyperparameters](https://docs.aws.amazon.com/sagemaker/latest/dg/object-detection-api-config.html). Here we selected a promising set of hyperparameters, after performing some initial testing. 

<div class="alert alert-block alert-warning">
<b>Important:</b> Our training dataset is quite small (only 66 images). If we train a large neural network from scratch, the results will not be good, because the model parameters will be more than the available training examples. As such, we are using <b>Transfer Learning</b>, by initializing our network with weights trained from a very large dataset (ImageNet) which contains millions of images. By doing so, we hope that our network will automatically learn some basic image features, like corners, edges and colors. Then, we use a <b>very low learning rate</b> (to avoid catastrophic forgetting), and we adjust the network weights only for a <b>few epochs</b>. This way, our network will adjust to our small dataset without forgetting the basic image features learned from millions of images. This technique is also known as <b>Supervised Finetuning</b>.
</div>

In [ ]:
estimator.set_hyperparameters(
    # Pre-training is particularly important for tiny data-sets like this!
    base_network="resnet-50",  # or 'vgg-16' architecture to be used
    use_pretrained_model=1,  # 0/1 whether to use pretrained or random weights (0-> train from scratch) 
    early_stopping = True,
    early_stopping_min_epochs = 10,
    early_stopping_patience = 5,
    early_stopping_tolerance = 0.00,
    num_classes=len(CLASS_NAMES),
    mini_batch_size=8,  # depends on the GPU memory and the image sizes
    epochs=35,  # only for a few epochs. 
    learning_rate=0.00009,  # very small learning rate to avoid catastrophic forgetting
    lr_scheduler_step="20,40,60,80",
    lr_scheduler_factor=0.5,
    optimizer="adam",
    momentum=0.99,
    weight_decay=0.98,
    overlap_threshold=0.5,
    nms_threshold=0.45,
    image_shape=832,
    label_width=350,
    num_training_samples=n_samples_training,
)


## Step 4: Train the model

The hyperparameters above represent our best up-front guess; and it's easy enough to call `estimator.fit()` to train a model as shown below.

One way to improve model performance and reduce some of the guesswork, is to let SageMaker `HyperParameterTuner` optimize them. SageMaker Hyper-Parameter Optiomization (HPO) supports [Random, Grid, Bayesian and Hyperband strategies](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-how-it-works.html)). In this example we prefer the [Hyperband approach, which usually exhibits a more competitive performance compared to ther rest](https://aws.amazon.com/blogs/machine-learning/amazon-sagemaker-automatic-model-tuning-now-provides-up-to-three-times-faster-hyperparameter-tuning-with-hyperband/). Because HPO typically takes much longer than standard model fitting, `tuner.fit()` is an **asynchronous** method by default whereas `estimator.fit()` is **synchronous** (blocking).

<div class="alert alert-block alert-warning">
<b>Notice:</b> The following training code will take <b>approximately 10 minutes</b> to execute (without HPO!). If you opt to activate HPO, it will take much longer!
</div>

In [ ]:
WITH_HPO = False  # change to True if you want to find better parameters (attention! it takes long time!)

In [ ]:
%%time

if WITH_HPO is False:
    estimator.fit(
        { "train": train_channel, "validation": validation_channel }, 
        logs=True
    )
    
else:
    hyperparameter_ranges = {
        "learning_rate": sagemaker.tuner.ContinuousParameter(0.00001, 0.01),
        "momentum": sagemaker.tuner.ContinuousParameter(0.0, 0.99),
        "weight_decay": sagemaker.tuner.ContinuousParameter(0.0, 0.99),
        "optimizer": sagemaker.tuner.CategoricalParameter(["sgd", "adam", "rmsprop", "adadelta"])
    }

    tuner = sagemaker.tuner.HyperparameterTuner(
        estimator,
        "validation:mAP",  # Name of the objective metric to optimize. "Mean Average Precision" high = good
        objective_type="Maximize",  # or Minimize
        strategy="Hyperband", # or Baysian or Random. 
        hyperparameter_ranges=hyperparameter_ranges,
        base_tuning_job_name="object-detection-SSD-HPO",
        max_jobs=20,  # how many searches (training jobs) we will have in the parameter space
        max_parallel_jobs=1  # how many searches (training jobs) will happen in parallel
    )
    
    tuner.fit(
        { "train": train_channel, "validation": validation_channel },
        include_cls_metadata=False
    )

<div class="alert alert-block alert-warning">
<b>Note:</b> While you are waiting for the training to finish, you may start exploring the 2nd notebook: <b>2.Train-augmented-object-detection-model.ipynb</b>, which performs augmentations on the initial small dataset, in order to increase its size.</div>

If you ever lose the notebook state e.g. due to a kernel restart or crash, you can **attach()** you estimator/tuner to a previous training/tuning job as follows (uncomment and run). There is no need to retrain because the results are all stored.

In [ ]:
# Examples to attach to a previous training run:

#estimator.attach("SSD-HPO-220924-1158-003-ba8e84f9")  # change to the name of the training job

#tuner.attach("SSD-HPO-220924-1158")  # change to the name of the HPO job

#WITH_HPO=?

Once training finishes, we can explore the training logs and see how performance on the validation set changed over time through the training epochs.


In [ ]:
import boto3
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%matplotlib inline
plt.style.use('seaborn')

client = boto3.client("logs")
BASE_LOG_NAME = "/aws/sagemaker/TrainingJobs"

def plot_object_detection_log(model, title):
    # retrieve from the training job logs the mAP across epochs and plot it
    logs = client.describe_log_streams(
        logGroupName=BASE_LOG_NAME, logStreamNamePrefix=model._current_job_name
    )
    cw_log = client.get_log_events(
        logGroupName=BASE_LOG_NAME, logStreamName=logs["logStreams"][0]["logStreamName"]
    )

    mAP_accs = []
    for e in cw_log["events"]:
        msg = e["message"]
        if "validation mAP <score>=" in msg:
            num_start = msg.find("(")
            num_end = msg.find(")")
            mAP = msg[num_start + 1 : num_end]
            mAP_accs.append(float(mAP))

    print(title)
    print("Maximum mAP: %f " % max(mAP_accs))

    fig, ax = plt.subplots()
    plt.xlabel("Epochs")
    plt.ylabel("Mean Avg Precision (mAP)")
    plt.title("Validation performance per training epoch")
    (val_plot,) = ax.plot(range(len(mAP_accs)), mAP_accs, label="mAP")
    plt.legend(handles=[val_plot])
    ax.yaxis.set_ticks(np.arange(0.0, 1.05, 0.1))
    ax.yaxis.set_major_formatter(ticker.FormatStrFormatter("%0.2f"))
    plt.show()
 
    
if WITH_HPO is True: 
    estimator = tuner.best_estimator()
    
plot_object_detection_log(estimator, "mAP tracking for job: " + estimator._current_job_name)

## Step 5: Deploy the model

Once the model is trained, SageMaker supports many different [deployment options](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html). In this example we'll deploy our trained model to a **real-time endpoint**, in order to have the lowest possible latency in our predictions. 

You can think of an endpoint as a dedicated web-server, making accessible our trained model's predictions through a REST API. Since our endpoints won't be handling any significant traffic volumes, we provision a single non-accelerated instance.

<div class="alert alert-block alert-warning">
<b>Notice:</b> The following deployment of the model to a realtime endpoint will take approximately <b>5-7 minutes</b>.
</div>

<div class="alert alert-block alert-warning">
<b>Attention:</b> Please copy the <b>name of the deployed endpoint</b> (in the output of the following cell). You will need to paste it in the application. This will allow the application to route camera images to your endpoint for real-time inference.
</div>

In [ ]:
%%time
if WITH_HPO is True:
    if (predictor_hpo):  # in case HPO was selected
        try:
            predictor_hpo.delete_endpoint()
            print("Deleted previous HPO endpoint")
        except:
            print("Couldn't delete previous HPO endpoint")
    print("Deploying HPO model...")
    predictor_hpo = tuner.deploy(
        initial_instance_count=1,
        instance_type="ml.c5.large",
        # wait=False,
    )
else:
    if (predictor_std):  # in case simple training was selected
        try:
            predictor_std.delete_endpoint()
            print("Deleted previous non-HPO endpoint")
        except:
            print("Couldn't delete previous non-HPO endpoint")
    print("Deploying standard (non-HPO) model...")
    predictor_std = estimator.deploy(
        initial_instance_count=1,  # number of instances for the endpoint
        instance_type="ml.c5.large",  # type of instance to be used for the endpoint
        # wait=False,
    )

predictor = predictor_hpo if WITH_HPO else predictor_std
print('Name of the deployed endpoint:',predictor.endpoint_name)

<div class="alert alert-block alert-warning">
<b>Attention:</b> Please copy the <b>name of the deployed endpoint</b> (in the output of the previous cell). You will need to paste it in the application. This will allow the application to route camera images to your endpoint for real-time inference.
</div>

## Step 6: Run inference on test images

Now that we have one object detection model deployed, we can send some test images and see how it performs!


#### Test on individual images

The `predict()` function used here, provided in the `util` folder, includes code that allows us to send image files to our deployed endpoint and receive back the detection results. 

The built-in Object Detection algorithm doesn't estimate an optimal confidence threshold for us. Instead, it returns **all the detections**, irrespective of their confidence score. The `predict()` function includes the confidence threshold parameter `thresh`. Any detection with confidence score higher than the `thresh` parameter, will be visualized. The function also returns a Pandas DataFrame with all the detected bounding boxes for the given detection threshold. 

For visualization, the `predict()` function calls the `visualize_detection()` function, which is uses Matplotlib to plot the provided detection boxes over the image. You can test different filenames from the test folder, as well as different `thresh` values.

In [ ]:
%load_ext autoreload
%autoreload 2

from util.util import predict

sagemaker_runtime = boto3.client(service_name="runtime.sagemaker")
endpoint_name = predictor.endpoint_name
print('Using endpoint:',endpoint_name)

df_results = predict(
    # change this to other test images to see more results
    filename=f'{LOCAL_DATASET_FOLDER}/dataset-test/IMG_1599.jpg',  
    runtime=sagemaker_runtime,
    class_names = CLASS_NAMES,
    endpoint_name=endpoint_name, 
    thresh=0.2, 
    visualize=True
)

df_results


#### Compare multiple detection thresholds

You can even generate side by side comparison of the results of multiple detection thresholds, by providing a **list of thresholds**, instead of a single number. In the following example, we depict the results for 3 different detection threshold values `[0.2, 0.4, 0.6]`.

In [ ]:
%load_ext autoreload
%autoreload 2

from util.util import predict

sagemaker_runtime = boto3.client(service_name="runtime.sagemaker")
endpoint_name = predictor.endpoint_name
print('Using endpoint:',endpoint_name)

df_results = predict(
    # change this to other test images to see more results
    filename=f'{LOCAL_DATASET_FOLDER}/dataset-test/IMG_1599.jpg',  
    runtime=sagemaker_runtime,
    class_names = CLASS_NAMES,
    endpoint_name=endpoint_name, 
    thresh=[0.2,0.4,0.6],  # a list of thresholds to be visualized
    visualize=True
)


#### Test on the whole test set

We can now call the `evaluate_testset()` helper function, which will test the deployed model against the **whole test set** and generate a performance report, for each class. The helper function relies on two important parameters. **Intersection Over Union (IOU)** and the model's confidence threshold. IOU is a metric that describes the degree of overlap between two bounding boxes, and it is needed in order to know when the output of the model coincides with the ground truth bounding box. Check this page for a more [detailed description on IOU](https://en.wikipedia.org/wiki/Jaccard_index).

In [ ]:
%load_ext autoreload
%autoreload 2

from util.util import evaluate_testset

df_class_performance = evaluate_testset(
    runtime=sagemaker_runtime, 
    endpoint_name=endpoint_name,
    class_names = CLASS_NAMES,
    testset_folder = f'{LOCAL_DATASET_FOLDER}/dataset-test', 
    test_manifest_file = f'{LOCAL_DATASET_FOLDER}/dataset-test/test-updated.manifest', 
    thr_iou = 0.5,  # Intersection Over Union threshold
    thr_conf = 0.2  # Confidence Threshold
)

df_class_performance


Here is a quick explanation for the performance statistics. Check this page for a more [detailed description](https://en.wikipedia.org/wiki/Precision_and_recall).

| Abbreviation | Full name | Explanation | Range | Intuition |
| --- | --- | --- | --- | --- | 
| `TP` | True Positives | How many times the model has correctly detected an actual object | Depends on test set | Higher is better |
| `FP` | False Positives | How many times the model has mistakenly detected something that was not an actual object | Depends on test set  | Lower is better |
| `FN` | False Negatives | How many times the model did not detect (missed) an actual object | Depends on test set | Lower is better |
| `TN` | True Negatives | (**Does not apply in object detection!**) How many times the model correctly did not detect something that was not an actual object | N/A | N/A |
| `PR` | Precision | What percentage of all the model's detections were indeed actual objects. | [0,1] | Higher is better |
| `RE` | Recall | What percentage of all the actual available objects were detected by the model | [0,1] | Higher is better |
| `F1` | F1 score | A balanced combination (harmonic mean) of Precision and Recall.  | [0,1] | Higher is better |

In [ ]:
# plot a graph of the results 

plt.figure()
df_class_performance.plot.bar(x='CLASS')
plt.title('Model performance on the Testset')
plt.show()


## Step 7: Clean up

Although training instances are ephemeral, the resources we allocated for real-time endpoints need to be cleaned up to avoid ongoing charges. The code below will delete the *most recently deployed* endpoint for the HPO and non-HPO configurations, but note that if you deployed either more than once, you might end up with extra endpoints. To be safe, it's best to still check through the SageMaker console for any left-over resources when cleaning up.

In [ ]:
# uncomment the following code in order to delete the created resources

# if (predictor_hpo):
#     print("Deleting HPO-optimized predictor endpoint")
#     predictor_hpo.delete_endpoint()
# if (predictor_std):
#     print("Deleting standard (non-HPO) predictor endpoint")
#     predictor_std.delete_endpoint()

## Review

In this notebook we used Amazon SageMaker to train a custom computer vision object detection model using the built-in Object Detection algorithm. We particularly used a **Tranfer Learning** technique, which allowed us to leverage pre-trained weights from a larger generic dataset and use a smaller specialized training dataset, relevant to our use case. We deployed the trained model in a real-time endpoint and we evaluated its performance on the hold-out dataset, as well as on a real-time camera feed. 
